In [1]:
%%html
<style>
.rendered_html *{
    text-align: left !important;
}
</style>

# Solution Structure

## Introduction
[Tutor](https://docs.tutor.overhang.io/index.html) is the Docker-based [Open edX distribution](https://open.edx.org/) [[docs](https://edx.readthedocs.io/projects/edx-developer-guide/en/latest/index.html)] [[guide](https://edx.readthedocs.io/projects/edx-developer-guide/en/latest/index.html)], both for production and local development. The goal of Tutor is to make it easy to deploy, customize, upgrade and scale Open edX.

**Tutor Links**
- [Github](https://github.com/overhangio/tutor)
- [Docs](https://docs.tutor.overhang.io/index.html)
- [Video](https://www.youtube.com/watch?v=p0NoqR8zlyw)

**Open eDx**
The links to other [components](##Components) and [plugins](##Plugins) are available on their respective sections.
- [Website](https://open.edx.org/)
- [Wiki](https://openedx.atlassian.net/wiki/spaces/OpenOPS/overview)
- [Docs](https://edx.readthedocs.io/projects/edx-developer-guide/en/latest/index.html)
- [Guide](https://edx.readthedocs.io/projects/edx-developer-guide/en/latest/index.html)


## Development
The entry point program for `tutor` is located at `./setup.py`.
```python
entry_points={"console_scripts": ["tutor=tutor.commands.cli:main"]},
```

All startup components are then initialized within the `cli.py`'s `main` function.
```python
def main() -> None:
    ...
```

### Pre-requisites
To build and package `tutor` locally:

1. Follow the steps in [DevSetup](./DevSetup.ipynb) to create and activate the `tutor` virtual environment with [venv](https://docs.python.org/3/library/venv.html).
2. Restore all project dependencies with `pip install -r ./requirements/dev.in`. 
3. Ensure that the following build tools are installed via `pip list`.
    - [setuptools](https://pypi.org/project/setuptools/)
    - [wheel](https://pypi.org/project/wheel/)
    - [twine](https://pypi.org/project/twine/)
4. Ensure the following files are available [before building the library](https://medium.com/analytics-vidhya/how-to-create-a-python-library-7d5aea80cc3f):

| Files | Description |
|---|---|
| setup.py | `setuptools` will read the library configuration from this file. |
| tutor | The application folder containing business logics. |
| tutor/__init__.py | Upon import of the library, this file will automatically get executed and discover all functions that are exposed from the library. |
| tests | Unit tests for your library |

### Build and Debug
1. After writing your changes, you can try to [install and debug](https://stackoverflow.com/a/69102148/3104587) `tutor` locally with `pip install -e .`, so it can still be edited in-place.
2. Setup neccessary breakpoints with either of the follwing ways:
    - In VSCode, set breakpoint manually on the left gutter.
    - Set breakpoints using python debug frameworks:
        - [debugpy](https://code.visualstudio.com/docs/python/debugging#_command-line-debugging): `debugpy.breakpoint()`
3. You can then try to debug your codes with one of the following ways:
    - In VSCode, run the `Python: Startup` or `Python: Current File` debug profile in the Run and Debug menu.
    - If you set breakpoints using debugging frameworks, then:
        - [debugpy](https://code.visualstudio.com/docs/python/debugging#_command-line-debugging): `python -m debugpy --listen 0.0.0.0:5678 --wait-for-client ./myscript.py`
        - Finally, attach the debugging framework process with your code editor, e.g. [VSCode](https://code.visualstudio.com/docs/python/debugging#_command-line-debugging).


### Packaging
To [package and distribute](https://packaging.python.org/guides/distributing-packages-using-setuptools/) `tutor` to either [Pypi](https://pypi.org/) or to your private repository. 
1. Build `tutor` via the command `python setup.py build` or `python setup.py bdist_wheel`. This should create the following folders:
    - build
    - tutor.egg-info
    - dist
2. Check the build manifests under `./build/lib` folder.
3. Try install the built package onto your virtual environment using `pip install './dist/tutor.whl'`.
4. Upload your library to Pypi to private repository using `python -m twine upload dist/*`.


### Testing
All tests should be written into the `./tests` folder. Currently, tutor uses the following python libraries for testing:
- pylint
- mypy
- black
- unittest (`python -m unittest discover tests`)

## Commands
In tutor, every commands stated in the `main` function earlier has its own dedicated module within the solution folder `./commands`. Those commands are basically thin wrappers around other tools, e.g. `tutor k8s` are wrapper to the `kubectl` command.

| Module | Description |
|---|:---|
| local | Run Open edX locally with docker-compose. |
| dev | Run Open edX locally with development settings. |
| k8s | Run Open edX on Kubernetes |
| print_help |  |
| config_command | Configure Open edX and store configuration values in $TUTOR_ROOT/config.yml. |
| images_command |  |
| plugins_command |  |


## Configuration
Each instance of `tutor` hosted will have their own configuration root.
```python
tutor config printroot
ls -la
```

### Global Configuration
Inside the configuration root folder, it will then contain various configurations for that `tutor` instance. With Tutor, all Open edX deployment parameters are stored in a single `config.yml` file.

| Path | Description |
|---|:---|
| data | Contain app configurations for each `tutor` application. |
| env | Contains environmental specific configurations. E.g. if tutor is hosted with `tutor k8s quickstart` command, all Kubernetes object configurations yaml will be contained here.  <table> <thead>   <tr>     <th>Path</th>     <th>Description</th>   </tr> </thead> <tbody>   <tr>     <td>data</td>     <td>Contains </td>   </tr>   <tr>     <td>env</td>     <td>Contains environmental specific configurations. E.g. if tutor is hosted with `tutor k8s quickstart` command, all Kubernetes object configurations yaml will be contained here.</td>   </tr>   <tr>     <td>config.yml</td>     <td>All Open Edx global configurations are stored here for the hosted instance.</td>   </tr> </tbody> </table> |
| config.yml | All Open Edx global configurations are stored here for the hosted instance. |

When resolving the global configuration values, `tutor` will follow the following priority:
- `config.yml`
- `defaults.config.yml`
- `user.config.yml`
- Environment variables (prefixed with `TUTOR_`)

With an up-to-date environment, Tutor is ready to launch an Open edX platform and perform usual operations. Below, we document some of the configuration parameters.

### Service Configuration
Every instance of `tutor` comes with a few built-in service that can be configured.

| Config | Description |
|---|:---|
| RUN_LMS | (default: true) |
| RUN_CMS | (default: true) |
| RUN_FORUM | (default: true) |
| RUN_ELASTICSEARCH | (default: true) |
| RUN_MONGODB | (default: true) |
| RUN_MYSQL | (default: true) |
| RUN_REDIS | (default: true) |
| RUN_SMTP | (default: true) |
| ENABLE_HTTPS | (default: false) |

### Docker Configuration

### LMS Configuration
| Config | Description |
|---|:---|
| LMS_HOST | |

## Template
The `tutor` templates are pre-defined configuration templates used for deployment. Depending on which environment `tutor` will be deployed to, the templates will used global configuration values from `config.yml` and store the final configuration on the `env` folder.

All templates are contained within the `./templates` folder:

| Path | Description |
|---|:---|
| apps | Configuration templates for each `tutor` depedencies. |
| build | Configuration templates for building `tutor`. |
| dev, local, k8s | Configuration templates for each deployment environments. |
| hooks | Additional container hooks that will run during deployment. |


## Plugin
Tutor comes with a [plugin](https://docs.tutor.overhang.io/plugins.html) system that allows anyone to customise the deployment of an Open edX platform very easily. The vision behind this plugin system is that users should not have to fork the Tutor repository to customise their deployments. 

### API
Plugins can affect the behaviour of Tutor at multiple levels. E.g., plugins can define new services with their Docker images, settings and the right initialisation commands. To do so you will have to define custom [config](https://docs.tutor.overhang.io/plugins/api.html#plugin-config), [patches](https://docs.tutor.overhang.io/plugins/api.html#plugin-patches), [hooks](https://docs.tutor.overhang.io/plugins/api.html#plugin-hooks) and [templates](https://docs.tutor.overhang.io/plugins/api.html#plugin-templates). Then, plugins can also extend the CLI by defining their own [commands](https://docs.tutor.overhang.io/plugins/api.html#plugin-command).

### YAML
[YAML files](https://docs.tutor.overhang.io/plugins/gettingstarted.html#yaml-file) that are stored in the tutor plugins root folder will be automatically considered as plugins. The location of the plugin root can be found by running:
```python
tutor plugins printroot
```

### Python
[Creating a plugin as a Python package](https://docs.tutor.overhang.io/plugins/gettingstarted.html#python-package) allows you to define more complex logic and to store your patches in a more structured way. Python Tutor plugins are regular Python packages that define a specific entrypoint: `tutor.plugin.v0`.

```python
from setuptools import setup
setup(
    ...
    entry_points={"tutor.plugin.v0": ["myplugin = myplugin.plugin"]},
)
```

## Theming

[Comprehensive theming](https://edx.readthedocs.io/projects/edx-installing-configuring-and-running/en/latest/ecommerce/theming.html) allows users to change the Open Edx platform appearence.

### Static Assets


## Dependencies
The following are some of the most used dependencies in tutor:

### Required
Note that pip does not use `requirements.yml` / `requirements.txt` when your project is installed as a dependency by others. Generally, for that, you will have to specify dependencies in the `install_requires` and `tests_require` arguments in your `setup.py` file.

```python
install_requires=load_requirements(),
```

### Development
| Component | Description |
|---|:---|
| [Appdirs](https://github.com/ActiveState/appdirs) | Defines OS agnostics app configuration directory paths |
| [Click](https://click.palletsprojects.com/) | Click is a Python package for creating beautiful command line interfaces in a composable way with as little code as necessary. |
| [Jinja](https://jinja.palletsprojects.com/en) | Jinja is a fast, expressive, extensible templating engine. Special placeholders in the template allow writing code similar to Python syntax. Then the template is passed data to render the final document. |
| [Mypy](https://mypy.readthedocs.io/en/stable/) | Mypy is a static type checker for Python 3 and Python 2.7. |
| [Pylint](https://pylint.org/) | Pylint is a Python static code analysis tool which looks for programming errors, helps enforcing a coding standard, sniffs for code smells and offers simple refactoring suggestions. |

## TODO

The following are the lists of TODO for the future development for `tutor` only:
- Use pytest instead of unittest module.